# Title:-------

### Introduction: 

There are many different test prep classes out there and at varying prices. these test prep courses all claim to help improve students' marks. **But can you determine if a student took a test preparation course based on their exam scores?** 

To determine if a student took a test prep course based off there marks from 3 exams we will use the data set ________ . The data set contains information about US high school students, and consists of the students gender, race/ethnicity, parental level of education, lunch, test preparation course attendance and  the students' marks for 3 exams in math, reading and writing. #The students marks my be influenced by any of the other varibles. (should this be included)#

**things needed in the Introduction:**
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question

In [2]:
#is there enough relevant background info to understand rest of proposal?
#Question is stated
#ID and describe the ds. is there anything else to be add? 

factors to consider does the student have outside tutoring, 

look at the race/ethnicity varible to see if could fine better info on it 


### Primary Exploratory Data Analysis:

**include:**

Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [ ]:
#find out from TA how big n should be. >1000? 

In [1]:
#read dataset from the wed 

In [3]:
#clean and wrangle data
#select for only exam marks and prep class

detrimine in any rows are missing data 

### Methods:

**include:**
Explain how you will conduct either your data analysis and which variables/columns you will use. 
Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results


### Expected outcomes and significance

**include:**
What do you expect to find?

What impact could such findings have?

What future questions could this lead to?